This notebook checks the expansion identities given in Section 2 and Appendix A in the paper.

Notation:

* $t$ is the target, $c$ is the center, $s$ is the source
* $p$ is the expansion order
* $\cos \theta = \frac{(t - c) \cdot (s - c)}{(|t - c| |s - c|}$

# Utilities

Define a utility function that checks for convergence of an identity.

The kernel is a function of the form $K(s, t)$.

The identity to be checked is a function of the form $Id(s, c, t, p)$.

In [1]:
import numpy as np
import numpy.linalg as la
import itertools


TESTS = 5
DIM = 3
ORDERS = np.array([3, 4, 5, 6, 7])


def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)


def verify_identity(knl, identity):
    np.random.seed(0)
    samples = 2 * np.random.random((3*TESTS, DIM)) - 1

    for s, c, t in grouper(samples, 3):
        if la.norm(t - c) > la.norm(s - c):
            s, t = t, s

        errors = []

        true_val = knl(s, t)

        for order in ORDERS:
            errors.append(np.max(np.abs(true_val - identity(s, c, t, order))))

        conv_factor = np.exp(np.polyfit(1 + ORDERS, np.log(errors), 1)[0])
        assert conv_factor < min(1, 1.3 * la.norm(t - c) / la.norm(s - c)), conv_factor

    print("Verified!")

And an auxiliary function to compute Legendre polynomial values.

In [2]:
def legvals(x, n):
    pjm2 = 1
    pjm1 = x

    vals = np.zeros(1 + n)
    derivs = np.zeros(1 + n)

    vals[0] = 1

    derjm2 = 0
    derjm1 = 1

    if n == 0:
        return vals, derivs

    vals[1] = x
    derivs[1] = 1

    if n == 1:
        return vals, derivs

    for j in range(2, n + 1):
        pj = ( (2*j-1)*x*pjm1-(j-1)*pjm2 ) / j
        vals[j] = pj

        derj = (2*j-1)*(pjm1+x*derjm1)-(j-1)*derjm2
        derj = derj / j
        derivs[j] = derj

        derjm2 = derjm1
        derjm1 = derj

        pjm2 = pjm1
        pjm1 = pj

    return vals, derivs

# Spherical Harmonics
The spherical harmonic addition theorem states that
$$ P_n(\cos \theta) = \frac{4 \pi}{2n+1} \sum_{m=-n}^n Y^m_n(\theta_x, \phi_x) \overline{Y^{m}_n(\theta_y, \phi_y)}. $$

In [3]:
def cart2polar(x):
    # Uses NumPy convention
    r = la.norm(x)
    theta = np.arctan2(x[1], x[0]) 
    phi = np.arccos(x[2] / r)
    return (r, theta, phi)

np.random.seed(0)
x, y = np.random.random((2, 3))

x /= la.norm(x)
y /= la.norm(y)

cos_theta = np.dot(x, y)
_, theta_x, phi_x = cart2polar(x)
_, theta_y, phi_y = cart2polar(y)

n = 5

lvals, _ = legvals(cos_theta, n)

from scipy.special import sph_harm

# This uses NumPy's spherical harmonics, which are different from those defined in
# the paper, but still satisfy the addition theorem.
sph_harm_x = sph_harm(np.arange(-n, 1+n), n, theta_x, phi_x)
sph_harm_y = sph_harm(np.arange(-n, 1+n), n, theta_y, phi_y)

assert np.allclose(np.vdot(sph_harm_x, sph_harm_y), lvals[-1] * (2*n+1) / (4*np.pi))

ModuleNotFoundError: No module named 'scipy'

# Laplace Identities

The Laplace Green's function $\mathcal{G}$:

$$ \mathcal{G}(t, s) = (4 \pi)^{-1} |t - s|^{-1}. $$

The target-specific expansion for the Laplace Green's function:

$$ \mathcal{G}(t, s) = (4 \pi)^{-1} \sum_{n=0}^\infty \frac{|t-c|^n}{|s-c|^{n+1}} P_n(\cos \theta). $$

In [4]:
def ts_l3d(s, c, t, p):
    r = la.norm(t - c)
    rho = la.norm(s - c)
    cos_theta = np.dot(s - c, t - c) / (r * rho)
    lvals, _ = legvals(cos_theta, p)

    return sum(
        (4 * np.pi) ** -1
        * r**k * rho**-(k+1)
        * lvals[k]
        for k in range(0, 1+p))

def k_l3d(s, t):
    return 1 / (4 * np.pi * la.norm(s - t))

verify_identity(k_l3d, ts_l3d)

Verified!


The $t$-gradient of the Laplace Green's function:
    
$$ \nabla_t \mathcal{G}(t, s) = -(4 \pi)^{-1} \frac{(t-s)}{|t-s|^{3}}. $$

The target-specific expansion for the $t$-gradient:

$$ \nabla_t \mathcal{G}(t, s) = (4 \pi)^{-1} \sum_{n=1}^\infty
\frac{|t-c|^{n-1}}{|s-c|^{n+1}}
\left(
n \frac{t - c}{|t-c|}
P_n(\cos \theta)
+
\left[
\frac{s-c}{|s-c|}
- \frac{t-c}{|t-c|} \cos(\theta)
\right]
P_n'(\cos \theta) \right)
. $$

In [5]:
def ts_tgrad_l3d(s, c, t, p):
    r = la.norm(t - c)
    rho = la.norm(s - c)
    cos_theta = np.dot(s - c, t - c) / (r * rho)
    lvals, derivs = legvals(cos_theta, p)
    
    return sum(
        (4 * np.pi) ** -1 * r**(k-1) * rho**-(k+1)
        * (
            k * (t - c) / r * lvals[k]
            + ((s - c) / rho - ((t - c) / r) * cos_theta) * derivs[k]
        ) for k in range(1, 1+p))


def k_tgrad_l3d(s, t):
    val = -(t-s) / (4*np.pi * la.norm(t-s)**3)
    return val

verify_identity(k_tgrad_l3d, ts_tgrad_l3d)

Verified!


The $s$-gradient of the Laplace Green's function:
    
$$ \nabla_s \mathcal{G}(t, s) = (4 \pi)^{-1} \frac{(t-s)}{|t-s|^{3}}. $$

The target-specific expansion for the $s$-gradient:

$$ \nabla_s \mathcal{G}(t, s) = (4 \pi)^{-1}
\sum_{n=0}^\infty
\frac{|t-c|^{n}}{|s-c|^{n+2}}
\left(
-(n+1) \frac{s - c}{|s - c|}
P_n(\cos \theta)
+
\left[
\frac{t-c}{|t-c|}
- \frac{s-c}{|s-c|} \cos(\theta)
\right]
P_n'(\cos \theta) \right)
. $$

In [6]:
def ts_sgrad_l3d(s, c, t, p):
    r = la.norm(t - c)
    rho = la.norm(s - c)
    cos_theta = np.dot(s - c, t - c) / (r * rho)
    lvals, derivs = legvals(cos_theta, p)
    
    return sum(
        (4 * np.pi) ** -1 * r**k * rho**-(k+2)
        * (
            -(k + 1) * (s - c) / rho * lvals[k]
            +((t - c) / r - ((s - c) / rho) * cos_theta) * derivs[k]
        ) for k in range(0, 1 + p))


def k_sgrad_l3d(s, t):
    val = (t-s) / (4*np.pi * la.norm(t-s)**3)
    return val

verify_identity(k_sgrad_l3d, ts_sgrad_l3d)

Verified!


 # Helmholtz Case
 
 The Green's function for the Helmholtz equation:
 $$ \mathcal{G}_k(t, s) = \frac{e^{ik|t-s|}}{4 \pi |t - s|}. $$
 
 The target-specific expansion for the Green's function:
 $$ \mathcal{G}_k(t, s) = (4 \pi)^{-1} ik \sum_{n=0}^\infty (2n+1) j_n(k|t-c|) h_n(k|s-c|) P_n(\cos \theta). $$

In [7]:
HELMHOLTZ_K = 3

from scipy.special import spherical_jn as jn, spherical_yn as yn

def hn(n, x, derivative=False):
    return jn(n, x, derivative) + 1j * yn(n, x, derivative)

def ts_h3d(s, c, t, p):
    r = la.norm(t - c)
    rho = la.norm(s - c)
    cos_theta = np.dot(s - c, t - c) / (r * rho)
    lvals, _ = legvals(cos_theta, p)

    return sum(
        (4 * np.pi) ** -1
        * 1j * HELMHOLTZ_K
        * (2*k + 1)
        * jn(k, HELMHOLTZ_K * r)
        * hn(k, HELMHOLTZ_K * rho)
        * lvals[k]
        for k in range(0, 1+p))

def k_h3d(s, t):
    x = la.norm(t - s)
    return np.exp(1j * HELMHOLTZ_K * x) / (4 * np.pi * x)

verify_identity(k_h3d, ts_h3d)

Verified!


The $t$-gradient of the Green's function:
$$\nabla_t \mathcal{G}_k(t, s) = 
(4\pi)^{-1} e^{ik|t-s|} \left( \frac{ik(t-s)}{|t-s|^2} - \frac{(t-s)}{|t-s|^3} \right).
$$

The target-specific expansion for the $t$-gradient:
$$\nabla_t \mathcal{G}_k(t, s) = 
(4 \pi)^{-1} ik \sum_{n=0}^\infty (2n+1) \frac{h_n(k|s-c|)}{|t-c|}
    \left( k(t-c)j_n'(k|t-c|)P_n(\cos \theta)
    + \left[ \frac{s-c}{|s-c|} - \frac{t-c}{|t-c|} \cos \theta \right]
    j_n(k|t-c|) P_n'(\cos \theta) \right).
$$

In [8]:
def ts_tgrad_h3d(s, c, t, p):
    r = la.norm(t - c)
    rho = la.norm(s - c)
    cos_theta = np.dot(s - c, t - c) / (r * rho)
    lvals, derivs = legvals(cos_theta, p)

    return sum(
        (4 * np.pi) ** -1
        * 1j * HELMHOLTZ_K
        * (2*k + 1)
        * hn(k, HELMHOLTZ_K * rho) / r
        * (
            HELMHOLTZ_K * (t - c) * jn(k, HELMHOLTZ_K * r, True) * lvals[k]
            + (
                ((s - c) / rho - (t-c) * cos_theta / r)
                * jn(k, HELMHOLTZ_K * r)) * derivs[k]
        ) for k in range(0, 1+p))

def k_tgrad_h3d(s, t):
    x = la.norm(t - s)
    return (
        1/(4*np.pi)
        * np.exp(1j * HELMHOLTZ_K * x)
        * (1j * HELMHOLTZ_K * (t - s) / x**2 - (t - s)/x**3))

verify_identity(k_tgrad_h3d, ts_tgrad_h3d)

Verified!


The $s$-gradient of the Green's function:
$$\nabla_s \mathcal{G}_k(t, s) = 
(4\pi)^{-1} e^{ik|t-s|} \left(\frac{(t-s)}{|t-s|^3} - \frac{ik(t-s)}{|t-s|^2}  \right).
$$

The target-specific expansion for the $s$-gradient:
$$\nabla_s \mathcal{G}_k(t, s) = 
(4 \pi)^{-1} ik \sum_{n=0}^\infty (2n+1) \frac{j_n(k|t-c|)}{|s-c|}
    \left( k(s-c)h_n'(k|s-c|)P_n(\cos \theta)
    + \left[ \frac{t-c}{|t-c|} - \frac{s-c}{|s-c|} \cos \theta \right]
    h_n(k|s-c|) P_n'(\cos \theta) \right).
$$

In [9]:
def ts_sgrad_h3d(s, c, t, p):
    r = la.norm(t - c)
    rho = la.norm(s - c)
    cos_theta = np.dot(s - c, t - c) / (r * rho)
    lvals, derivs = legvals(cos_theta, p)

    return sum(
        (4 * np.pi) ** -1
        * 1j * HELMHOLTZ_K
        * (2*k + 1)
        * jn(k, HELMHOLTZ_K * r) / rho
        * (
            HELMHOLTZ_K * (s - c) * hn(k, HELMHOLTZ_K * rho, True) * lvals[k]
            + (
                ((t - c) / r - (s-c) * cos_theta / rho)
                * hn(k, HELMHOLTZ_K * rho)) * derivs[k]
        ) for k in range(0, 1+p))

def k_sgrad_h3d(s, t):
    x = la.norm(t - s)
    return (
        1/(4*np.pi)
        * np.exp(1j * HELMHOLTZ_K * x)
        * (((t - s)/x**3) - 1j * HELMHOLTZ_K * (t - s) / x**2))

verify_identity(k_sgrad_h3d, ts_sgrad_h3d)

Verified!
